# Booking to EZskin

This project is to book a clinic through python selenium

Author : Wake Liu

<table>
<td>
<img src="pic1.png" style="width:300;height:450px;">

</td>

</table>

- clock_setting: 若為False，直接執行該程式
- times:重刷次數
- interval等待時間

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import time

clock_setting = False
hour = 23
minute = 59
sec = 59
times = 3
interval = 0.6
username = "F227268068"
password = "lovewake2018"

#### 由於每個ID都是獨立的，我們可以在頁面中找到自己要的ID資訊。
#### 以EZskin為例，在登入時所需要的資料為ID, Password, Login, 從網站原始碼截取的資訊如下

In [2]:
def login_proc(username, password):
    login_url = "https://www.ezskin.com.tw/member/member_login.aspx"
    sel = webdriver.Chrome()
    sel.get("https://www.ezskin.com.tw/member/member_login.aspx")
    # sign in the username
    try:
        user_input = sel.find_element_by_id("ctl00_ContentPlaceHolder1_txtUserID")
        user_input.clear()
        user_input.send_keys(username)
        print ('user-id write success!')
    except:
        print ('user-id write error!')
    # sign in the pasword
    try:
        pwd_input = sel.find_element_by_id("ctl00_ContentPlaceHolder1_txtPassword")
        pwd_input.clear()
        pwd_input.send_keys(password)
        print ('pw write success!')
    except:
        print ('pw write error!')
    # Check for Login success
    try:
        sel.find_element_by_id("ctl00_ContentPlaceHolder1_btnLogin").click()
    except:
        print ('Login Failed')

    while True:
        curpage_url = sel.current_url
        if curpage_url != login_url:
            if curpage_url[:-1] != login_url:  # choose wrong verify_pic
                print ('Login finished!')
                break
        else:
            time.sleep(5)
            print ('Login Failed!')
    return sel

In [3]:
sel = login_proc(username, password)

user-id write success!
pw write success!
Login finished!


#### Since 門診掛號沒有id,這邊用Xpath的方式將該element 找到

In [4]:
try:
    sel.find_element_by_xpath('//*[@id="body"]/div[4]/div[2]/div[1]/div/div[2]/table/tbody/tr/td/table/tbody/tr[4]/td/div/input').click()
    print ("準備門診掛號...")
except BaseException:
    print ("無法門診掛號...")

準備門診掛號...


#### 仔細觀察，該checkbox有固定的格式

#### 如果有出現該id就直接click然後break 離開迴圈

In [5]:
def select_proc():
    flag = 0
    try:
        sel.refresh()
        #a = 1
    except:
        print ("Cannot Access this website")
    current_date = sel.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_dbOutpatient"]/tbody/tr[9]/td[1]').text
    if date != current_date:
        print (current_date,"booking is opening...")
    for i in range (1,4):
        checkbox = 'ctl00_ContentPlaceHolder1_dbOutpatient_ctl09_chkSelect'+str(i)
        try:   
            sel.find_element_by_id(checkbox).click()
            flag = 1
            break
        except BaseException:
            continue
            #print(checkbox + " element not exist")
    return flag

In [6]:
date = sel.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_dbOutpatient"]/tbody/tr[9]/td[1]').text

#### 這個步驟是等待，我們只有在23時59分59秒才需要開啟該程式，其他時間都可以休息

In [7]:
while(clock_setting):
    localtime = time.localtime(time.time())
    #print ("localtime :", localtime)
    if localtime.tm_hour == hour and localtime.tm_min == minute and localtime.tm_sec == sec:
        while(1):
            localtime = time.localtime(time.time())
            if localtime.tm_sec == (sec+1)%60 :
                sleep(interval)
                break
        break
    print (localtime.tm_hour,localtime.tm_min,localtime.tm_sec)
    if localtime.tm_hour != hour:
        sleep(3600)
    elif localtime.tm_min != minute:
        sleep(60)
    else:
        sleep(1)



#### 等待

In [8]:
current_time = time.time()
for i in range(times):
    flag = select_proc()
    time_cost = time.time()-current_time
    current_time = time.time()
    if flag == True:
        print ("Successfully Selected, the time is i = ",i, time.localtime(current_time),time_cost)
        break
    else :
        print ("Keep Waiting, the time is i = ",i, time.localtime(current_time),time_cost)
        

Keep Waiting, the time is i =  0 time.struct_time(tm_year=2018, tm_mon=3, tm_mday=13, tm_hour=21, tm_min=31, tm_sec=37, tm_wday=1, tm_yday=72, tm_isdst=0) 0.5802969932556152
Keep Waiting, the time is i =  1 time.struct_time(tm_year=2018, tm_mon=3, tm_mday=13, tm_hour=21, tm_min=31, tm_sec=37, tm_wday=1, tm_yday=72, tm_isdst=0) 0.6020090579986572
Keep Waiting, the time is i =  2 time.struct_time(tm_year=2018, tm_mon=3, tm_mday=13, tm_hour=21, tm_min=31, tm_sec=38, tm_wday=1, tm_yday=72, tm_isdst=0) 0.600775957107544


#### 下一步跟確認掛號都算是簡單的，如果有成功掛號(即flag為true)，只要抓到id繼續click就可以了

##### 先點擊下一步
<input type="submit" name="ctl00$ContentPlaceHolder1$btnNext" value="下一步" onclick="return checkModify();" id="ctl00_ContentPlaceHolder1_btnNext" class="btn01">

##### 確認掛號語法

這邊還是要碰一點運氣，有機會鍛羽而歸

<input type="submit" name="ctl00$ContentPlaceHolder1$btnSave" value="確定掛號" id="ctl00_ContentPlaceHolder1_btnSave" class="btn01">

In [9]:
if flag == True:
    try:
        sel.find_element_by_id("ctl00_ContentPlaceHolder1_btnNext").click()
        print ("Successfully Next Step!")
    except BaseException:
        print ("Cannot Press Next step!")
    try:
        sel.find_element_by_id("ctl00_ContentPlaceHolder1_btnSave").click()
        print ("Successfully Saved!")
    except BaseException:
        print ("Cannot save")
        
    msg = sel.switch_to_alert().text
    if '您已經在此門診掛號' in msg:
        print (msg,"Congratulation!!! Your reservation is done!")
    else:
        print (msg,"Sorry >_< all the doctors are reserved!")
    sel.switch_to_alert().accept()
else:
    print ("Sorry >_< all the doctors are reserved!")

Sorry >_< all the doctors are reserved!


In [10]:
localtime = time.localtime(time.time())
print ("Current Time is"+str(localtime))

Current Time istime.struct_time(tm_year=2018, tm_mon=3, tm_mday=13, tm_hour=21, tm_min=31, tm_sec=38, tm_wday=1, tm_yday=72, tm_isdst=0)
